# Importing the Required Libraries 

In [1]:
# loading the reqed libraries for scraping from Twitter and Reddit 
import tweepy 
import praw
import os  
from dotenv import load_dotenv
import torch 
import pandas as pd
import praw


In [5]:
# Loading the env varibles for the twitter and Reddit 
load_dotenv()
twit_api= os.getenv("TWIT_API")
twit_secret = os.getenv("TWIT_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

# Grabbing the Twitter Data

In [6]:

# authentication
auth = tweepy.OAuthHandler(twit_api, twit_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


# tweets = tweepy.Cursor(api.user_timeline, screen_name=user, count=200, tweet_mode='extended').items(limit)

# search tweets
keywords = 'BTC'
limit=300

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=10, tweet_mode='extended').items(limit)

# tweets = api.user_timeline(screen_name=user, count=limit, tweet_mode='extended')

# create DataFrame
columns = ['User', 'Tweet']
data = []

for tweet in tweets:
    data.append([tweet.user.screen_name, tweet.full_text])

df = pd.DataFrame(data, columns=columns)

print(df)
df.to_csv('BTC-300.csv')

                User                                              Tweet
0         kka69dri2b  I'm not jealous. I'm not jealous. I'm really j...
1        AxienRaider  RT @BenkoTrades: Positive PA on $BTC\n\nGot a ...
2    simonnwintodayx  RT @Heather99924565: Wow #2112Run is doing a l...
3              TrSnR  RT @SplashyWhales: 📈💶$6,544,892 $BTC LONGED @$...
4         gronharder  RT @BattleKnightICO: 💰 How to buy BKN token?👇👇...
..               ...                                                ...
295     stefancarlos  RT @RicardoBSalinas: ¿Cuando vamos a vender #b...
296    yellowholloww  RT @LukaEvans_: The sun goes down… not like th...
297      tbooncho888  btc day ema20 and ema50 hv turned green....on ...
298         Dree_Btc  RT @aivatars_nft: @3hmedplus @psychedelic_nft ...
299    CAJanuszewski  A master crypto trader just hodles. Nothing mo...

[300 rows x 2 columns]


In [7]:
import praw

In [10]:
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_SEC"),
    password=os.getenv("REDDIT_PASS"),
    user_agent="stock_dashboard by u/CryptoFoDayz",
    username=os.getenv("REDDIT_USER"),
)


reddit.read_only = True


In [11]:
for subreddit in reddit.subreddits.default(limit=10):
    print(subreddit)

gadgets
sports
gaming
pics
worldnews
videos
AskReddit
aww
Music
funny


# Perform Sentiment Analysis 

In [2]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# create tokenizer variable to chose the model 
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")



Downloading: 100%|██████████| 638M/638M [00:15<00:00, 43.5MB/s] 


In [15]:
df.drop(columns='User')


,Tweet
0,I'm not jealous. I'm not jealous. I'm really jealous.😭😭#BTC #cryptocurrency #Bitcoin https://t.co/CiWP6kXFYd
1,RT @BenkoTrades: Positive PA on $BTC\n\nGot a running flat as opposed to expanded flat proposed below leading to the breakout. Still on the s…
2,RT @Heather99924565: Wow #2112Run is doing a launch party today @ 4:20pm EST!!! Don't miss it!\n\n#ETH #ethereum #NFT #NFTCommunity #nfts #cr…
3,"RT @SplashyWhales: 📈💶$6,544,892 $BTC LONGED @$43932.0 [08/02/22 00:02:29]\n🏪FTX | BTC-PERP\n\n🗣️As Abraham Lincoln once said, Fuck it - etcmin…"
4,"RT @BattleKnightICO: 💰 How to buy BKN token?👇👇\n🌐 Go to your personal account on the site https://t.co/M15fPfYKuK\n➡️ Go to the ""Wallet"" sec…"
...,...
295,RT @RicardoBSalinas: ¿Cuando vamos a vender #btc en la App de @BancoAzteca? https://t.co/WWG44bSs6S
296,"RT @LukaEvans_: The sun goes down… not like the market that is going up after a few weeks 📈🚀\n\nAfter being super low, market is on fire and…"
297,btc day ema20 and ema50 hv turned green....on its way to 300k wave 5 top? https://t.co/Vrg9IPdkvQ
298,RT @aivatars_nft: @3hmedplus @psychedelic_nft @AzukiZen @ThePixelmon @InvsbleFriends @WGMInterfaces @WGMI_Industries @coffeeclannft Super e…


In [20]:
# encode the list of tweet to create the sentiment 
for item in df: 
    tokens = tokenizer.encode(item)




AttributeError: 'list' object has no attribute 'shape'

In [18]:
result = model(tokens)


AttributeError: 'list' object has no attribute 'size'

In [ ]:
result.logits